# Toward patch-free:
## Patch Maker
We will create larger & larger patches till our CNN runs out of GPU RAM.

In [1]:
import time
import os
import glob
from PIL import Image
Image.MAX_IMAGE_PIXELS = None
import numpy as np
import json
import tensorflow as tf
from tensorflow import keras
import keras.layers as kl
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
DIR_IMAGES_RAW = "/home/jrm/Martinez/images/raw/"
DIR_IMAGES_TRAIN = "/home/jrm/Martinez/images/BigPatch/train/"
DIR_IMAGES_VALID = "/home/jrm/Martinez/images/BigPatch/valid/"
PATTERN_IMAGES_RAW = "*.DF1.*.tif"
PATCH_SIZE=224  # 224 matches VGG
PATCH_SIZE=256  
PATCH_SIZE=512     # dies
#PATCH_SIZE=1024   # dies
# PATCH_SIZE=2048   # kernel death during training
IMAGE_SIZE = (PATCH_SIZE,PATCH_SIZE)
DF_Ypos = ['B7.','B15','D1.','D5.','E7.','E9.','F9.','G3.','H13','I1.','I5.','I13']
DF_Yneg = ['A3.','A5.','B13','C1.','C11','D3.','E5.','F3.','F7','F11','F13','F15','G15','H1.','H3.','H7.','H15']

In [3]:
def get_image_names(path,pattern):
    paths = glob.glob(path+pattern)
    names = [os.path.basename(x) for x in paths]
    return names
FILENAMES_IMAGES_RAW = get_image_names(DIR_IMAGES_RAW,PATTERN_IMAGES_RAW)

In [4]:
class patch_maker:
    def __init__(self):
        self.path=""
        self.w=0
        self.h=0
        self.im_width = 0
        self.im_height = 0
        self.patch_size = 10 # scalar, assumed square for now
        self.pixel_array = None
        self.output_dir = ""
        self.output_format='.jpg'
    def set_input_path(self,path):
        self.path=path
    def set_output_path(self,path):
        self.output_dir = path
    def set_patch_size(self,scalar):
        self.patch_size = scalar
    def load_pixel_array(self,filename,verbose=False):
        self.w=0
        self.h=0
        im = Image.open(self.path+filename)
        ima = np.array(im)   # convert to numpy
        self.im_width = ima.shape[0]
        self.im_height = ima.shape[1]    
        if verbose:
            print(filename, ima.size, ima.shape)
        self.pixel_array = ima
        return ima
    def get_next_patch(self): 
        IM_WIDTH = self.im_width
        IM_HEIGHT = self.im_height
        PIXEL_ARRAY = self.pixel_array
        patch = None
        if self.w+self.patch_size > IM_WIDTH:
            self.h += self.patch_size
            self.w = 0
        if self.w+self.patch_size <= IM_WIDTH and self.h+self.patch_size <= IM_HEIGHT:
            patch = PIXEL_ARRAY[self.w:self.w+self.patch_size, self.h:self.h+self.patch_size]
            self.w += self.patch_size
        return patch
    def save_patch (self, prefix, pnum, patch):
        ext = self.output_format
        path= self.output_dir+prefix+'.'+str(pnum)+ext
        im =  Image.fromarray(patch)
        im.save(path)

In [5]:
pm = patch_maker()
pm.set_input_path(DIR_IMAGES_RAW)
pm.set_patch_size(PATCH_SIZE)
pm.set_output_path(DIR_IMAGES_TRAIN)  
for filename in FILENAMES_IMAGES_RAW:
    print(filename,end=" ")
    file_counts = {'Yneg':0, 'Ypos':0}
    neg_or_pos = 0
    prefix = filename[0:3]
    if prefix[2] == '.':  # we have prefixes like F1. and F13
        prefix=prefix[0:2]+'_'   # avoid the extra period
    neg_or_pos = +1
    phenotype = 'Ypos'
    if prefix in DF_Yneg:
        neg_or_pos = -1
        phenotype = 'Yneg'
    pixels = pm.load_pixel_array(filename)
    patch = pm.get_next_patch()
    while patch is not None:
        file_counts[phenotype] += 1
        pm.save_patch(phenotype+'/'+prefix,file_counts[phenotype],patch)
        patch = pm.get_next_patch()
    print(file_counts)

F15.DF1.135.tif {'Yneg': 576, 'Ypos': 0}
D5.DF1.43.tif {'Yneg': 0, 'Ypos': 576}
H3.DF1.27.tif {'Yneg': 0, 'Ypos': 576}
C11.DF1.96.tif {'Yneg': 576, 'Ypos': 0}
H13.DF1.114.tif {'Yneg': 0, 'Ypos': 576}
I1.DF1.09.tif {'Yneg': 0, 'Ypos': 576}
G3.DF1.25.tif {'Yneg': 0, 'Ypos': 576}
C1.DF1.01.tif {'Yneg': 0, 'Ypos': 576}
F11.DF1.94.tif {'Yneg': 576, 'Ypos': 0}
F9.DF1.80.tif {'Yneg': 0, 'Ypos': 576}
H7.DF1.62.tif {'Yneg': 0, 'Ypos': 576}
A3.DF1.26.tif {'Yneg': 0, 'Ypos': 576}
A5.DF1.45.tif {'Yneg': 0, 'Ypos': 576}
H15.DF1.132.tif {'Yneg': 576, 'Ypos': 0}
D1.DF1.02.tif {'Yneg': 0, 'Ypos': 576}
G15.DF1.133.tif {'Yneg': 576, 'Ypos': 0}
B15.DF1.134.tif {'Yneg': 0, 'Ypos': 576}
I5.DF1.48.tif {'Yneg': 0, 'Ypos': 576}
E9.DF1.75.tif {'Yneg': 0, 'Ypos': 576}
I13.DF1.117.tif {'Yneg': 0, 'Ypos': 576}
F3.DF1.22.tif {'Yneg': 0, 'Ypos': 576}
B7.DF1.60.tif {'Yneg': 0, 'Ypos': 576}
B13.DF1.115.tif {'Yneg': 576, 'Ypos': 0}
E7.DF1.64.tif {'Yneg': 0, 'Ypos': 576}
F7.DF1.63.tif {'Yneg': 0, 'Ypos': 576}
D3.DF1.20

In [6]:
# Move a portion of the train files into the valid directory.
from random import sample
VALIDATION_PORTION = 0.20
for phenotype in ('Ypos','Yneg'):
    fromdir = DIR_IMAGES_TRAIN+"/"+phenotype+"/"
    todir   = DIR_IMAGES_VALID+"/"+phenotype+"/"
    files = os.listdir(fromdir)
    total = len(files)
    selected = sample(files, int(VALIDATION_PORTION*total))
    for vfile in selected:
        os.rename(fromdir+vfile,todir+vfile)
print("Done")

Done
